In [1]:
from __future__ import print_function
import numpy as np

d = 64                           # dimension
nb = 1000000                      # database size
nq = 10000                       # nb of queries
np.random.seed(1234)             # make reproducible
xb = np.random.random((nb, d)).astype('float32')
xb[:, 0] += np.arange(nb) / 1000.
xq = np.random.random((nq, d)).astype('float32')
xq[:, 0] += np.arange(nq) / 1000.

import faiss

nlist = 100
k = 4
quantizer = faiss.IndexFlatL2(d)  # the other index
index = faiss.IndexIVFFlat(quantizer, d, nlist, faiss.METRIC_L2)
# here we specify METRIC_L2, by default it performs inner-product search

assert not index.is_trained
index.train(xb)
assert index.is_trained

index.add(xb)                  # add may be a bit slower as well
D, I = index.search(xq, k)     # actual search
print(I[-5:])                  # neighbors of the 5 last queries
index.nprobe = 10              # default nprobe is 1, try a few more
D, I = index.search(xq, k)
print(I[-5:])                  # neighbors of the 5 last queries

Failed to load GPU Faiss: No module named swigfaiss_gpu
Faiss falling back to CPU-only.


[[10179  9864  8813  9881]
 [ 9762 10013  9720 10405]
 [ 9229  9510 10039 10602]
 [ 9725  9787  9816 11091]
 [ 9464  9635 10199  9180]]
[[10179 10714  9864  8813]
 [ 9762 10013  9720 10405]
 [ 9229  9510 10039 10602]
 [10584  9725  9787  9816]
 [ 9464  9635 10199  9180]]


In [2]:
faiss.write_index(index, '2-IVFFlat.index')

In [3]:
loaded = faiss.read_index('2-IVFFlat.index')

In [4]:
D, I = loaded.search(xq, k)     # actual search
print(I[-5:])                  # neighbors of the 5 last queries

[[10179 10714  9864  8813]
 [ 9762 10013  9720 10405]
 [ 9229  9510 10039 10602]
 [10584  9725  9787  9816]
 [ 9464  9635 10199  9180]]


In [5]:
loaded.nprobe = 10              # default nprobe is 1, try a few more
D, I = loaded.search(xq, k)
print(I[-5:])                  # neighbors of the 5 last queries

[[10179 10714  9864  8813]
 [ 9762 10013  9720 10405]
 [ 9229  9510 10039 10602]
 [10584  9725  9787  9816]
 [ 9464  9635 10199  9180]]
